In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
planes_df = spark.read.csv("file:/databricks/driver/datasets/planes.csv",inferSchema=True,header=True)
planes_df.show(truncate=False)
planes_df.count()

+-------+----+-----------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|type                   |manufacturer    |model   |engines|seats|speed|engine   |
+-------+----+-----------------------+----------------+--------+-------+-----+-----+---------+
|N102UW |1998|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |NA   |Turbo-fan|
|N103US |1999|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |NA   |Turbo-fan|
|N104UW |1999|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |NA   |Turbo-fan|
|N105UW |1999|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |NA   |Turbo-fan|
|N107US |1999|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |NA   |Turbo-fan|
|N108UW |1999|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |NA   |Turbo-fan|
|N109UW |1999|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |NA   |Turbo-fan|
|N110UW |1999|Fixed wing multi engine|AIRBUS INDUS

In [0]:
airport_df = spark.read.csv("file:/databricks/driver/datasets/airports.csv",inferSchema=True,header=True)
airport_df.show(truncate=False)
airport_df.count()

+---+------------------------------------+----------------+-----------------+----+---+---+
|faa|name                                |lat             |lon              |alt |tz |dst|
+---+------------------------------------+----------------+-----------------+----+---+---+
|04G|Lansdowne Airport                   |41.1304722      |-80.6195833      |1044|-5 |A  |
|06A|Moton Field Municipal Airport       |32.4605722      |-85.6800278      |264 |-5 |A  |
|06C|Schaumburg Regional                 |41.9893408      |-88.1012428      |801 |-6 |A  |
|06N|Randall Airport                     |41.431912       |-74.3915611      |523 |-5 |A  |
|09J|Jekyll Island Airport               |31.0744722      |-81.4277778      |11  |-4 |A  |
|0A9|Elizabethton Municipal Airport      |36.3712222      |-82.1734167      |1593|-4 |A  |
|0G6|Williams County Airport             |41.4673056      |-84.5067778      |730 |-5 |A  |
|0G7|Finger Lakes Regional Airport       |42.8835647      |-76.7812318      |492 |-5 |A  |

In [0]:
flight_dtls_df =  spark.read.csv("file:/databricks/driver/datasets/flights_small.csv",inferSchema=True,header=True)
flight_dtls_df.show(truncate=False)
flight_dtls_df.count()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|12   |8  |658     |-7       |935     |-5       |VX     |N846VA |1780  |SEA   |LAX |132     |954     |6   |58    |
|2014|1    |22 |1040    |5        |1505    |5        |AS     |N559AS |851   |SEA   |HNL |360     |2677    |10  |40    |
|2014|3    |9  |1443    |-2       |1652    |2        |VX     |N847VA |755   |SEA   |SFO |111     |679     |14  |43    |
|2014|4    |9  |1705    |45       |1839    |34       |WN     |N360SW |344   |PDX   |SJC |83      |569     |17  |5     |
|2014|3    |9  |754     |-1       |1015    |1        |AS     |N612AS |522   |SEA   |BUR |127     |937     |7   |54    |
|2014|1    |15 |1037    |7        |1352 

In [0]:
origin_df = flight_dtls_df.join(planes_df, on = [planes_df['tailnum']==flight_dtls_df['tailnum']],how='inner')\
.join(airport_df, on = [airport_df['faa']==flight_dtls_df['origin']],how='inner')
origin_df.show(truncate=False)
origin_df.count()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------+----+-----------------------+--------------+-----------+-------+-----+-----+---------+---+-------------------+---------+-----------+---+---+---+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|tailnum|year|type                   |manufacturer  |model      |engines|seats|speed|engine   |faa|name               |lat      |lon        |alt|tz |dst|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------+----+-----------------------+--------------+-----------+-------+-----+-----+---------+---+-------------------+---------+-----------+---+---+---+
|2014|12   |8  |658     |-7       |935     |-5       |VX     |N846VA |1780  |SEA   |LAX |132     |954     |6   |58    |N846VA |2011|Fixed wing multi engine|AIRBUS        |A320-214   |2

In [0]:
dest_df = flight_dtls_df.join(planes_df, on = [planes_df['tailnum']==flight_dtls_df['tailnum']],how='inner')\
.join(airport_df, on = [airport_df['faa']==flight_dtls_df['dest']],how='inner')
dest_df.show(truncate=False)
dest_df.count()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------+----+-----------------------+--------------+-----------+-------+-----+-----+---------+---+----------------------------------+---------+-----------+----+---+---+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|tailnum|year|type                   |manufacturer  |model      |engines|seats|speed|engine   |faa|name                              |lat      |lon        |alt |tz |dst|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------+----+-----------------------+--------------+-----------+-------+-----+-----+---------+---+----------------------------------+---------+-----------+----+---+---+
|2014|12   |8  |658     |-7       |935     |-5       |VX     |N846VA |1780  |SEA   |LAX |132     |954     |6   |58    |N846VA |2011|Fixe

In [0]:
final_df = origin_df.unionAll(dest_df)
final_df.show(truncate=False)
final_df.count()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------+----+-----------------------+--------------+-----------+-------+-----+-----+---------+---+-------------------+---------+-----------+---+---+---+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|tailnum|year|type                   |manufacturer  |model      |engines|seats|speed|engine   |faa|name               |lat      |lon        |alt|tz |dst|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------+----+-----------------------+--------------+-----------+-------+-----+-----+---------+---+-------------------+---------+-----------+---+---+---+
|2014|12   |8  |658     |-7       |935     |-5       |VX     |N846VA |1780  |SEA   |LAX |132     |954     |6   |58    |N846VA |2011|Fixed wing multi engine|AIRBUS        |A320-214   |2

In [0]:
final_df.groupBy(col('name').alias('Airport')).agg(count('flight').alias("Number of Flights"))\
.orderBy(col("Number of Flights").desc())\
.show(truncate=False)

+-------------------------------+-----------------+
|Airport                        |Number of Flights|
+-------------------------------+-----------------+
|Seattle Tacoma Intl            |6481             |
|Portland Intl                  |3261             |
|San Francisco Intl             |772              |
|Los Angeles Intl               |612              |
|Denver Intl                    |574              |
|Mc Carran Intl                 |520              |
|Phoenix Sky Harbor Intl        |517              |
|Ted Stevens Anchorage Intl     |442              |
|Salt Lake City Intl            |395              |
|Norman Y Mineta San Jose Intl  |369              |
|Metropolitan Oakland Intl      |333              |
|Sacramento Intl                |283              |
|Chicago Ohare Intl             |271              |
|San Diego Intl                 |270              |
|Hartsfield Jackson Atlanta Intl|258              |
|Minneapolis St Paul Intl       |236              |
|George Bush

In [0]:
final_df.groupBy(col('name').alias('Airport'),col('flight').alias("Flight Number"))\
.agg(count('flight').alias("Flight Flown"))\
.show(truncate=False)

+-------------------+-------------+------------+
|Airport            |Flight Number|Flight Flown|
+-------------------+-------------+------------+
|Seattle Tacoma Intl|656          |26          |
|Seattle Tacoma Intl|1473         |14          |
|Portland Intl      |381          |3           |
|Portland Intl      |384          |13          |
|Seattle Tacoma Intl|1632         |6           |
|Seattle Tacoma Intl|147          |4           |
|Seattle Tacoma Intl|334          |3           |
|Seattle Tacoma Intl|1675         |1           |
|Portland Intl      |4688         |1           |
|Portland Intl      |403          |4           |
|Seattle Tacoma Intl|1494         |1           |
|Seattle Tacoma Intl|1542         |23          |
|Seattle Tacoma Intl|3677         |5           |
|Portland Intl      |1584         |2           |
|Seattle Tacoma Intl|1704         |1           |
|Portland Intl      |1414         |4           |
|Seattle Tacoma Intl|294          |3           |
|Seattle Tacoma Intl

### Questions on Busiest Airport

In [0]:
busiest_airport_df = final_df.groupBy(col('name').alias('Airport')).agg(count('flight').alias("Number of Flights"))\
.orderBy(col("Number of Flights").desc())\
.limit(1)

busiest_airport_df.show(truncate=False)

+-------------------+-----------------+
|Airport            |Number of Flights|
+-------------------+-----------------+
|Seattle Tacoma Intl|6481             |
+-------------------+-----------------+



In [0]:
# Extract the busiest airport name
busiest_airport_name = busiest_airport_df.select("Airport").first()[0]

window_spec = Window.orderBy(desc("Occurrences"))

max_flown_flight_df = (
    final_df
    .filter(col('name') == busiest_airport_name)
    .groupBy('flight')
    .agg(count('flight').alias("Occurrences"))
    .withColumn("dense_rank", dense_rank().over(window_spec))
    .filter(col("dense_rank") == 1)  # Select rows with rank 1 (max occurrences)
    .drop("dense_rank")
)
max_flown_flight_df.show()

+------+-----------+
|flight|Occurrences|
+------+-----------+
|   614|         33|
|   664|         33|
|   616|         33|
+------+-----------+



In [0]:
# Extract the busiest airport name
busiest_airport_name = busiest_airport_df.select("Airport").first()[0]

window_spec = Window.orderBy(desc("Occurences"))

busiest_airport_year = (
    final_df.groupBy(col('name').alias('Airport'),planes_df['year'])
    .agg(count('flight').alias("Occurences"))
    .orderBy(col("Occurences").desc())
    .withColumn("dense_rank", dense_rank().over(window_spec))
    .filter(col("dense_rank") == 1)  # Select rows with rank 1 (max occurrences)
    .drop("dense_rank")
)

busiest_airport_year.show(truncate=False)

+-------------------+----+----------+
|Airport            |year|Occurences|
+-------------------+----+----------+
|Seattle Tacoma Intl|2001|581       |
+-------------------+----+----------+



In [0]:
# Combine 'year', 'month', and 'day' columns to create a 'date' column
planes_flown_time = flight_dtls_df.withColumn("arr_time", to_timestamp(concat(col("year"), lit("-"), col("month"), lit("-"), col("day"), lit(" "), col("hour"), lit(":"), col("minute"))))

planes_flown_time.select('flight','tailnum','origin','dest','arr_time','air_time').show(truncate=False)

+------+-------+------+----+-------------------+--------+
|flight|tailnum|origin|dest|arr_time           |air_time|
+------+-------+------+----+-------------------+--------+
|1780  |N846VA |SEA   |LAX |2014-12-08 06:58:00|132     |
|851   |N559AS |SEA   |HNL |2014-01-22 10:40:00|360     |
|755   |N847VA |SEA   |SFO |2014-03-09 14:43:00|111     |
|344   |N360SW |PDX   |SJC |2014-04-09 17:05:00|83      |
|522   |N612AS |SEA   |BUR |2014-03-09 07:54:00|127     |
|48    |N646SW |PDX   |DEN |2014-01-15 10:37:00|121     |
|1520  |N422WN |PDX   |OAK |2014-07-02 08:47:00|90      |
|755   |N361VA |SEA   |SFO |2014-05-12 16:55:00|98      |
|490   |N309AS |SEA   |SAN |2014-04-19 12:36:00|135     |
|26    |N564AS |SEA   |ORD |2014-11-19 18:12:00|198     |
|448   |N323AS |SEA   |LAX |2014-11-08 16:53:00|130     |
|656   |N305AS |SEA   |PHX |2014-08-03 11:20:00|154     |
|608   |N433AS |SEA   |LAS |2014-10-30 08:11:00|127     |
|121   |N765AS |SEA   |ANC |2014-11-12 23:46:00|183     |
|306   |N713AS

### Questions on Airlines Punctuality

In [0]:
airline_performance = (
    final_df.groupBy("carrier")
    .agg(
        avg("dep_delay").alias("avg_dep_delay"),
        avg("arr_delay").alias("avg_arr_delay")
    )
)

airline_performance.show()

+-------+------------------+-------------------+
|carrier|     avg_dep_delay|      avg_arr_delay|
+-------+------------------+-------------------+
|     UA| 9.944111776447105|  2.982017982017982|
|     AA|16.733333333333334| 10.733333333333333|
|     B6| 7.148325358851674|  2.813397129186603|
|     DL| 6.466604823747681| 1.4372093023255814|
|     OO| 3.553062985332183| 1.9644405897658284|
|     F9| 6.516666666666667|  5.438888888888889|
|     US| 2.692957746478873|-1.7859154929577465|
|     HA| 2.958904109589041| 2.2816901408450705|
|     AS|3.0360769641902725| 0.6581905781584583|
|     VX| 7.161290322580645|  1.564516129032258|
|     WN|12.071377072819034| 6.0592913955169925|
+-------+------------------+-------------------+



In [0]:
on_time_departures_percentage = (
    final_df.groupBy("carrier")
    .agg((count(when(col("dep_delay") <= 0, "dep_delay")) / count("*")).alias("on_time_departures_percentage"))
    .orderBy(col("on_time_departures_percentage").desc())
    .limit(1)
)

on_time_departures_percentage.show()

+-------+-----------------------------+
|carrier|on_time_departures_percentage|
+-------+-----------------------------+
|     HA|           0.8082191780821918|
+-------+-----------------------------+



In [0]:
on_time_arrivals_percentage = (
    final_df.groupBy("carrier")
    .agg((count(when(col("arr_delay") <= 0, "arr_delay")) / count("*")).alias("on_time_arrivals_percentage"))
    .orderBy(col("on_time_arrivals_percentage").desc())
    .limit(1)
)

on_time_arrivals_percentage.show()

+-------+---------------------------+
|carrier|on_time_arrivals_percentage|
+-------+---------------------------+
|     AA|         0.7777777777777778|
+-------+---------------------------+

